In [1]:
import pandas as pd
import numpy as np

In [2]:
loans = pd.read_csv("loans_py.csv")

In [3]:
accounts = pd.read_csv("../data/accounts.csv")
cards = pd.read_csv("../data/cards.csv")
districts = pd.read_csv("../data/districts.csv")
links = pd.read_csv("../data/links.csv")
payment_orders = pd.read_csv("../data/payment_orders.csv")
transactions = pd.read_csv("../data/transactions.csv")

/var/folders/qd/2fpgzg4s3vzc43qj0fq464wr0000gn/T/ipykernel_15207/3816154057.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv("../data/transactions.csv")


In [4]:
accounts = accounts.rename(columns = {"id":"account_id"})
accounts = accounts.rename(columns = {"date":"open_date"})
districts = districts.rename(columns = {"id":"district_id"})
accounts = accounts.merge(districts.loc[:,["district_id","name"]], how='outer')
accounts = accounts.rename(columns = {"name":"district_name"})
accounts = accounts.drop(columns="district_id", axis=1)

account_id: Account number\
district_name: Name of the district where the account is\
open_date: Date when account was opened\
statement_frequency: The frequency that statements are generated for the account\
loan: T/F if the account has a loan\
loan_amount: The amount of the loan if there is one, NA if none\
loan_payments: The amount of the loan payment if there is one, NA if none\
loan_term: The duration of loan in months, NA if none\
loan_status: The status of the loan (current or expired), NA if none\
loan_default: T/F if the loan is in default, or NA if none\
max_balance: Maximum balance in the account\
min_balance: Minimum balance in the account\

num_customers: The total number of clients associated with the account (owner and users)\
credit_cards: Number of credit cards for an account or zero if none\

max_withdrawal: Maximum amount withdrawn for the account\
min_withdrawal: Minimum amount withdrawn for the account\
cc_payments: Count of credit payments for the account for all cards\

In [5]:
loans = loans.drop(columns=["id","date"], axis=1)
default = []
status = []
for i in loans["status"]:
    if i == "A":
        status.append("expired")
        default.append(False)
    if i == "B":
        status.append("expired")
        default.append(True)
    if i == "C":
        status.append("current")
        default.append(False)
    if i == "D":
        status.append("current")
        default.append(False)
loans["status"] = status
loans["defualt"] = default
loans = loans.rename(columns={"amount":"loan_amount","payments":"loan_payments","term":"loan_term","status":"loan_status","defualt":"loan_default"})
loans

,account_id,loan_amount,loan_payments,loan_term,loan_status,loan_default
0,2,80952,3373,24,expired,False
1,19,30276,2523,12,expired,True
2,25,30276,2523,12,expired,False
3,37,318480,5308,60,current,False
4,38,110736,2307,48,current,False
...,...,...,...,...,...,...
677,11327,39168,1632,24,current,False
678,11328,280440,4674,60,current,False
679,11349,419880,6998,60,current,False
680,11359,54024,4502,12,expired,False


In [6]:
accounts = accounts.merge(loans, how='outer')

In [7]:
for i in accounts.index:
    if accounts.loc[i,"loan_amount"].is_integer():
        accounts.loc[i,"loan"] = True
    else:
        accounts.loc[i,"loan"] = False
accounts

,account_id,open_date,statement_frequency,district_name,loan_amount,loan_payments,loan_term,loan_status,loan_default,loan
0,1,1995-03-24,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False
1,343,1994-04-26,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False
2,413,1996-07-09,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False
3,431,1996-11-05,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False
4,568,1995-03-28,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...
4495,7231,1993-03-14,monthly,Praha - vychod,NaN,NaN,NaN,NaN,NaN,False
4496,7997,1996-04-25,monthly,Praha - vychod,160656.0,6694.0,24.0,current,False,True
4497,9104,1994-01-21,monthly,Praha - vychod,172080.0,3585.0,48.0,expired,False,True
4498,5650,1995-01-08,after every transaction,Praha - vychod,98592.0,4108.0,24.0,expired,False,True


In [8]:
links = links.drop(columns = "id")
cards = cards.drop(columns = ["id","type"])
cards = cards.rename(columns={"link_id":"client_id"})

In [9]:
a = links.merge(cards, how="outer")
b = pd.crosstab(index = a["account_id"], columns="num_customers").reset_index()
accounts = accounts.merge(b, how='outer')
accounts

,account_id,open_date,statement_frequency,district_name,loan_amount,loan_payments,loan_term,loan_status,loan_default,loan,num_customers
0,1,1995-03-24,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1
1,343,1994-04-26,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1
2,413,1996-07-09,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1
3,431,1996-11-05,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1
4,568,1995-03-28,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1
...,...,...,...,...,...,...,...,...,...,...,...
4495,7231,1993-03-14,monthly,Praha - vychod,NaN,NaN,NaN,NaN,NaN,False,1
4496,7997,1996-04-25,monthly,Praha - vychod,160656.0,6694.0,24.0,current,False,True,1
4497,9104,1994-01-21,monthly,Praha - vychod,172080.0,3585.0,48.0,expired,False,True,1
4498,5650,1995-01-08,after every transaction,Praha - vychod,98592.0,4108.0,24.0,expired,False,True,1


In [10]:
c = pd.crosstab(index = a.dropna(subset=['issue_date'])["account_id"], columns="credit_cards").reset_index()
accounts = accounts.merge(c, how='outer')
accounts['credit_cards'] = accounts['credit_cards'].fillna(0)
accounts

,account_id,open_date,statement_frequency,district_name,loan_amount,loan_payments,loan_term,loan_status,loan_default,loan,num_customers,credit_cards
0,1,1995-03-24,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1,0.0
1,343,1994-04-26,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1,0.0
2,413,1996-07-09,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1,0.0
3,431,1996-11-05,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1,1.0
4,568,1995-03-28,monthly,Pisek,NaN,NaN,NaN,NaN,NaN,False,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4495,7231,1993-03-14,monthly,Praha - vychod,NaN,NaN,NaN,NaN,NaN,False,1,0.0
4496,7997,1996-04-25,monthly,Praha - vychod,160656.0,6694.0,24.0,current,False,True,1,0.0
4497,9104,1994-01-21,monthly,Praha - vychod,172080.0,3585.0,48.0,expired,False,True,1,0.0
4498,5650,1995-01-08,after every transaction,Praha - vychod,98592.0,4108.0,24.0,expired,False,True,1,0.0


In [11]:
withdraw = transactions[transactions["type"]=="debit"]
payments = withdraw[withdraw["method"]=="credit card"]

for i in transactions["account_id"].unique():
    min_balance = transactions[transactions["account_id"]==i]["balance"].min()
    max_balance = transactions[transactions["account_id"]==i]["balance"].max()
    accounts.loc[accounts["account_id"]==i,"max_balance"] = max_balance
    accounts.loc[accounts["account_id"]==i,"min_balance"] = min_balance
    
    min_amount = withdraw[withdraw["account_id"]==i]["amount"].min()
    max_amount = withdraw[withdraw["account_id"]==i]["amount"].max()
    accounts.loc[accounts["account_id"]==i,"max_withdrawal"] = max_amount
    accounts.loc[accounts["account_id"]==i,"min_withdrawal"] = min_amount

    accounts.loc[accounts["account_id"]==i,"cc_payments"] = len(payments[payments["account_id"]==i])

In [12]:
accounts = accounts[['account_id', 'district_name', 'open_date', 'statement_frequency','num_customers','credit_cards', 'loan', 'loan_amount','loan_payments', 'loan_term', 'loan_status', 'loan_default','max_withdrawal', 'min_withdrawal', 'cc_payments', 'max_balance', 'min_balance']]
accounts = accounts.sort_values(by='account_id', ascending=True)
accounts

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan,loan_amount,loan_payments,loan_term,loan_status,loan_default,max_withdrawal,min_withdrawal,cc_payments,max_balance,min_balance
0,1,Pisek,1995-03-24,monthly,1,0.0,False,NaN,NaN,NaN,NaN,NaN,7500.0,15.0,0.0,30415.0,1000.0
46,2,Hl.m. Praha,1993-02-26,monthly,2,0.0,True,80952.0,3373.0,24.0,expired,False,42000.0,15.0,0.0,69302.0,1100.0
600,3,Kolin,1997-07-07,monthly,2,0.0,False,NaN,NaN,NaN,NaN,NaN,7400.0,15.0,0.0,53447.0,1000.0
665,4,Pribram,1996-02-21,monthly,1,0.0,False,NaN,NaN,NaN,NaN,NaN,5250.0,15.0,0.0,34870.0,800.0
707,5,Cesky Krumlov,1997-05-30,monthly,1,0.0,False,NaN,NaN,NaN,NaN,NaN,5100.0,15.0,0.0,32036.0,600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2614,11333,Mlada Boleslav,1994-05-26,monthly,1,0.0,False,NaN,NaN,NaN,NaN,NaN,62800.0,0.0,0.0,137043.0,-1291.0
599,11349,Hl.m. Praha,1995-05-26,weekly,2,0.0,True,419880.0,6998.0,60.0,current,False,64800.0,30.0,0.0,113679.0,200.0
3905,11359,Trebic,1994-10-01,monthly,1,0.0,True,54024.0,4502.0,12.0,expired,False,30100.0,15.0,22.0,81706.0,1000.0
2276,11362,Bruntal,1995-10-14,monthly,1,0.0,True,129408.0,5392.0,24.0,expired,False,16100.0,15.0,0.0,56056.0,1000.0
